In [7]:

import pandas as pd

import xml.etree.ElementTree as ET
import psycopg2
#required for navigating machine's directory
import glob
import os.path

#required for communicating with SQL database
from sqlalchemy import create_engine

from utilities import insert_to_table


In [24]:
conn_string  = "postgresql://myuser:mypassword@/postgres-container/mydatabase"
rawdata_path = "../rawdata/"
staging_data_dir = '../staging_data'

# E: Extracting data from multiple sources

In [14]:

"""
extract from a given file type: csv, json, xml
"""

def extract_from_csv(file_to_process: str) -> pd.DataFrame:
    dataframe = pd.read_csv(file_to_process)
    return dataframe

def extract_from_json(file_to_process: str) -> pd.DataFrame:
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

def extract_from_xml(file_to_process: str) -> pd.DataFrame:
    dataframe = pd.DataFrame(columns = columns)
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        sample = pd.DataFrame({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, index = [0])
        dataframe = pd.concat([dataframe, sample], ignore_index=True)
    return dataframe

"""
extract from folder
"""

def extract(folder,column) -> pd.DataFrame:
    extracted_data = pd.DataFrame(columns = columns)
    #for csv files
    for csv_file in glob.glob(os.path.join(folder, "*.csv")):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csv_file)], ignore_index=True)
    #for json files
    for json_file in glob.glob(os.path.join(folder, "*.json")):
        extracted_data = pd.concat([extracted_data, extract_from_json(json_file)], ignore_index=True)
    #for xml files
    for xml_file in glob.glob(os.path.join(folder, "*.xml")):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xml_file)], ignore_index=True)
    return extracted_data


In [15]:

columns = ['car_model','year_of_manufacture','price', 'fuel']
data = extract(rawdata_path, columns)

/tmp/ipykernel_118/74698113.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted_data = pd.concat([extracted_data, extract_from_csv(csv_file)], ignore_index=True)
/tmp/ipykernel_118/74698113.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataframe = pd.concat([dataframe, sample], ignore_index=True)
/tmp/ipykernel_118/74698113.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude e

In [16]:
data

,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.731343,Petrol
1,ciaz,2015,10223.880597,Diesel
2,ciaz,2015,11194.029851,Petrol
3,ertiga,2015,9104.477612,Petrol
4,dzire,2009,3358.208955,Petrol
...,...,...,...,...
85,etios liva,2014,7089.552239,Diesel
86,innova,2017,29477.611940,Petrol
87,fortuner,2010,13805.970149,Diesel
88,corolla altis,2011,6492.537313,Petrol


In [25]:
insert_to_table(data = data, conn_string = conn_string, table_name = 'extracted_cars')


OperationalError: (psycopg2.OperationalError) connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

# T: Transformation data and save organized data to .parquet file 

In [26]:
def transform(table_name):
    db = create_engine(conn_string)

    df = pd.read_sql_query(f'SELECT * FROM {table_name}',con=db)

    print(f"Shape of data {df.shape}")

    # truncate price
    df['price'] = round(df.price, 2)

    # remove samples with same car_model
    df = df.drop_duplicates(subset=['car_model'], keep='first')

    print(f"Shape of data {df.shape}")

    # write to parquet
    df.to_parquet(os.path.join(staging_data_dir, staging_file))




In [27]:
table_name = 'extracted_cars'
staging_file = 'car.parquet'
transform(table_name)

OperationalError: (psycopg2.OperationalError) connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

# L: Loading data

In [ ]:
def load(staging_data_dir) -> pd.DataFrame:
    data = pd.DataFrame()
    for parquet_file in glob.glob(os.path.join(staging_data_dir, "*.parquet")):
        data = pd.concat([pd.read_parquet(parquet_file),data])

    insert_to_table(data = data, conn_string = conn_string, table_name = 'ml_car_data')

    return data

data = load(staging_data_dir)
print(data.shape)

In [28]:
data

,car_model,year_of_manufacture,price,fuel
0,alto 800,2017,4253.731343,Petrol
1,ciaz,2015,10223.880597,Diesel
2,ciaz,2015,11194.029851,Petrol
3,ertiga,2015,9104.477612,Petrol
4,dzire,2009,3358.208955,Petrol
...,...,...,...,...
85,etios liva,2014,7089.552239,Diesel
86,innova,2017,29477.611940,Petrol
87,fortuner,2010,13805.970149,Diesel
88,corolla altis,2011,6492.537313,Petrol


# Check what table are in the database

In [29]:
query = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public';
    """

db = create_engine(conn_string)

pd.read_sql_query(query,con=db)

OperationalError: (psycopg2.OperationalError) connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: No such file or directory
	Is the server running locally and accepting connections on that socket?

(Background on this error at: https://sqlalche.me/e/20/e3q8)